#1 INICIALIZAR BIBLIOTECAS inicializar();

In [202]:
def inicializar():
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import pandas_datareader.data as pdr
    import yfinance as yf
    import datetime
    from datetime import date
    from pandas.tseries.offsets import BDay
    import winsound
    from time import time
    yf.pdr_override()
    return

LER CSV df=lercsv('nome');

In [201]:
#Ler csv
def lercsv(nome):
    pd.set_option('display.max_rows', 10)

    df = pd.DataFrame();

    df = pd.read_csv(nome)

    df.columns = ['tickers']

    #df = df.melt(id_vars=['empresa'])

    #df.drop(columns=['variable','empresa'],inplace=True)

    df.dropna(inplace=True)

    df= df.sort_values('tickers')

    df.index = range(df.shape[0])

    return df

BAIXAR DADOS df,aux = baixardados(ndias);

In [234]:
#baixar dados do yahoofinance
# df,aux = baixardados(ndias)
def baixardados(df,ndias):
    aux = []
    #baixar os dados do yahoofinance ultimos 15 dias uteis

    start = date.today() - BDay(ndias)
    for i in range(df.shape[0]):
        aux.append(pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=start));

    df.insert(1, 'dados', aux, allow_duplicates=True);

    print('======= ACABOU ===========')
    return df,aux

In [235]:
df = lercsv('B3.csv')
df,aux = baixardados(df,15)
winsound.Beep(690,420)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

SALVAR DF salvardf();

In [203]:
# salvar data frame
def salvardf():
    df.to_hdf('df.h5', 'obj1');
    return

#2 CARREGAR DF df,aux = carregardf();

In [204]:
# carregar data frame
def carregardf():
    df = pd.DataFrame();
    aux = [];
    df = pd.read_hdf('df.h5', 'obj1');
    for i in range(df.shape[0]-1,-1,-1):
        if df.iloc[i,1].shape[0] <= 3:
            df = df.drop(df.index[i])
    df.index = range(df.shape[0])
    for i in range(df.shape[0]):
        aux.append(df.iloc[i,1])
    return(df,aux)

#3 ATUALIZAR DF DIARIAMENTE aux = atualizacaodiaria(df,aux);

In [242]:
# atualização diária
# aux = atualizacaodiaria(df,aux)
def atualizacaodiaria(df,aux):

    #t1 = time()
    #for i in range(df.shape[0]):
    #    if (df.iloc[i,1].shape[0]>0):
    #        if (max(df.iloc[i,1].index) > data):
    #            data = max(df.iloc[i,1].index)

    for i in range(df.shape[0]):
        df2 = pd.DataFrame();
        if (max(df.iloc[i,1].index) == (date.today() - BDay(1))):
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=date.today()-BDay(1));
            aux[i] = pd.concat([aux[i],df2]);
        else:
            df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start=max(df.iloc[i,1].index)-BDay(1));
            aux[i] = pd.concat([aux[i],df2]);
            
        aux[i] = aux[i][~aux[i].index.duplicated(keep='last')]
        #if aux[i].shape[0] < 3:
        #    df = df.drop(df.index[i])
    #t2 = time() - t1
    
    #winsound.Beep(690,500)
    #print('Tempo decorrido: {} segundos'.format(t2))
    return(aux)

#4 SETUP 92 setup92(df,aux);

In [207]:
# função setup 9.2 #1
    
def setup92(df,aux):    
    
    
    
    #calculo da MME9
    tend = pd.DataFrame();
    for i in range(df.shape[0]):
        aux[i]['MME9'] = aux[i].Close.ewm(span=9).mean().dropna()
        aux[i]["SLOPE"]="";
        for k in range (1,aux[i].shape[0]):
            tend = aux[i].MME9[k] - aux[i].MME9[k-1]
            aux[i].iat[k,7]=tend    
    #perca de minima
    for i in range(df.shape[0]):
        aux[i]['mark_min'] = ''
        for j in range(1,aux[i].shape[0]):
           aux[i].loc[aux[i].index[j],'mark_min'] = np.where(((aux[i].Close[j] < aux[i].Low[j-1]) & (aux[i].SLOPE[j] > 0)),1,0)

    #superação de maxima
    for i in range(df.shape[0]):
        aux[i]['mark_max'] = ''
        for j in range(3,aux[i].shape[0]):
           aux[i].loc[aux[i].index[j],'mark_max'] = np.where(((aux[i].Close[j] > aux[i].High[j-1]) | (aux[i].Open[j] > aux[i].High[j-1])) & ((aux[i].mark_min[j] == 1) | (aux[i].mark_min[j-1] == 1) | (aux[i].mark_min[j-2] == 1) | (aux[i].mark_min[j-3] == 1)) ,1,0)

    #quantos mark_min
    contador_min=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_min[j] == 1):
                contador_min = contador_min + 1;

    #saber quantos mark_max
    contador_max=0
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):
            if(aux[i].mark_max[j] == 1):
                contador_max = contador_max + 1;

    #pegar todos os tickers com mark_max e as datas
    tickersmax = [];
    datamax = [];
    tickersmin = [];
    datamin = [];
    for i in range(df.shape[0]):
        for j in range(aux[i].shape[0]):

            if (aux[i]['mark_max'][j] == 1):
                tickersmax.append(df.iloc[i,0]);
                datamax.append(aux[i].index[j]);
            if (aux[i]['mark_min'][j] == 1):
                tickersmin.append(df.iloc[i,0]);
                datamin.append(aux[i].index[j]);
    #alocar os resultados em dois pd            
    df_mark_max = pd.DataFrame({'datas':datamax, 'tickers':tickersmax});
    df_mark_min = pd.DataFrame({'datas':datamin, 'tickers':tickersmin});
    #tirar valores duplicados ficando com o mais recente

    df_mark_max = df_mark_max.sort_values('datas').drop_duplicates('tickers',keep='last');
    df_mark_min = df_mark_min.sort_values('datas').drop_duplicates('tickers',keep='last');
    #setup 9.2 armado          
    df_setup_92_armado = pd.DataFrame();
    duplicates = set(df_mark_min.iloc[:,1]).intersection(df_mark_max.iloc[:,1]);
    df_setup_92_armado = df_mark_min[~df_mark_min['tickers'].isin(duplicates)]
    
    #tirar os valores antigos >5 dias
    for i in range(df_setup_92_armado.shape[0]-1,-1,-1):
        diff = abs(df_setup_92_armado['datas'].iloc[-1] - df_setup_92_armado.datas.iloc[i]);
        if diff.days > 7:
            df_setup_92_armado = df_setup_92_armado.drop([df_setup_92_armado.index[i]])
    #reindexar o indice       
    
    df_setup_92_armado.index = range(df_setup_92_armado.shape[0])
    df_setup_92_armado.tickers.to_csv('setup92.csv', index=False,line_terminator=',')
    
    return 

#5 GAPPINGUP gappingup(df,aux);

In [192]:
#gappingup(df,aux)
def gappingup (df,aux):
    gap_up = []
    for i in range(df.shape[0]):
        if ((aux[i].Open[aux[i].shape[0]-1] / aux[i].Close[aux[i].shape[0]-2]) > 1.01):
            gap_up.append(df.iloc[i,0])
    gap_up = pd.Series(gap_up)

    gap_up.to_csv('gap_up.csv', index=False,line_terminator=',')
    return

#6 GAPPINGDOWN gappingdown(df,aux);

In [193]:
#gappingdown(df,aux)
def gappingdown (df,aux):
    gap_down = []
    for i in range(df.shape[0]):
        if ((aux[i].Open[aux[i].shape[0]-1] / aux[i].Close[aux[i].shape[0]-2]) < 0.99):
            gap_down.append(df.iloc[i,0])
    gap_down = pd.Series(gap_down)

    gap_down.to_csv('gap_down.csv', index=False,line_terminator=',')
    return

#1 RODAR GERAL

In [244]:
t1 = time();
inicializar();
df,aux = carregardf();
aux = atualizacaodiaria(df,aux);
setup92(df,aux);
gappingup(df,aux);
gappingdown(df,aux);
t2 = time() - t1;
winsound.Beep(690,500)
print('Tempo decorrido: {} segundos'.format(t2))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

RETORNA INDEX DO TICKER print(index('ticker'))

In [218]:
#retorna index em dataframe
#print(df_setup_92_armado[df_setup_92_armado['tickers'] == 'ALSO3'].index.values)
def index(ticker):
    index = (df[df['tickers'] == ticker].index.values)
    return index

In [224]:
index('WIZS3')

array([267], dtype=int64)

In [238]:
i=0;
df2 = pdr.get_data_yahoo(df.iloc[i,0]+'.sa',start='2021-05-11');
aux[i] = pd.concat([aux[i],df2])

[*********************100%***********************]  1 of 1 completed


In [247]:
len(aux)

269

In [250]:
aux[268]

,Open,High,Low,Close,Adj Close,Volume,MME9,SLOPE,mark_min,mark_max
Date,,,,,,,,,,
2021-04-20,31.000000,31.090000,29.450001,29.549999,29.085804,4228000,29.549999,,,
2021-04-22,29.799999,30.030001,29.150000,29.510000,29.046432,2305600,29.527778,-0.022222,0,
2021-04-23,29.780001,30.080000,29.330000,29.590000,29.125177,1815000,29.553279,0.025501,0,
2021-04-26,29.620001,30.540001,29.129999,29.320000,28.859417,1837600,29.474255,-0.079024,0,0
2021-04-27,29.490000,29.910000,29.090000,29.650000,29.184233,1996700,29.526535,0.05228,0,0
...,...,...,...,...,...,...,...,...,...,...
2021-05-06,29.100000,29.100000,27.980000,28.389999,28.389999,1237900,29.067410,-0.185267,0,0
2021-05-07,28.450001,28.910000,28.190001,28.610001,28.610001,744400,28.970606,-0.096804,0,0
2021-05-10,28.590000,29.160000,28.430000,28.520000,28.520000,1543100,28.876339,-0.094267,0,0


In [234]:
teste=pd.DataFrame();
#print(df_setup_92_armado[df_setup_92_armado['tickers'] == 'ENMT4'].index.values)
df_setup_92_armado = df_setup_92_armado.drop(index=25)


In [222]:
index('RANI3')

array([206], dtype=int64)

In [266]:
linhas[1]

2

In [264]:
#data = {['ticker':['ABCD4','BCDE3','CDEF4'],'setor1':['IBOV','IBXX','IFNC'],'setor2':['IBXX','IBOV','ICON']]}
role = pd.DataFrame({'ticker':['ABCD4','BCDE3','CDEF4'],'setor1':['IBOV','IBXX','IFNC'],'setor2':['IBXX','ICON','IBOV']})
linhas = role.loc[(role.setor1 == 'IBOV') | (role.setor2 == 'IBOV')].index
print(linhas)

Int64Index([0, 2], dtype='int64')


In [131]:
#dataframe to zip 92armado
#lista de tickers pro profit line_terminator=','
#compression_opts = dict(method='zip',archive_name='setup92armado.csv')  
df_setup_92_armado.tickers.to_csv('setup92armado.csv', index=False,line_terminator=',')

In [130]:
#dataframe to zip gap_up
#lista de tickers pro profit line_terminator=','
#compression_opts = dict(method='zip',archive_name='gap_up.csv')  
gap_up.to_csv('gap_up.csv', index=False,line_terminator=',')

In [ ]:
#preços intraday
for i in range(df_setup_92_armado.shape[0]):
    for j in range(df.shape[0]):
        if (df_setup_92_armado.iloc[i,1]) == (df.iloc[j,0]):
            if intradayaux[i].shape[0] != 0:
                if (intradayaux[i].Close[intradayaux[i].shape[0]-1]) >= (aux[j].High[aux[j].shape[0]-1]) :
                    print (df_setup_92_armado.iloc[i,1])
                    
#watchlist
intradayaux = []
#if (df_setup_92_armado['dados']):
  #  del df_setup_92_armado['dados']
for i in range(df_setup_92_armado.shape[0]):
    #startdate = df_setup_92_armado.iloc[i,0];
    today = date.today();
    tomorrow = date.today() + datetime.timedelta(days=1)
    intradayaux.append(pdr.get_data_yahoo(df_setup_92_armado.iloc[i,1]+'.sa',start= today,end = tomorrow,interval="5m"));
    
df_setup_92_armado.insert(2, 'dados', intradayaux, allow_duplicates=True);

df_setup_92_armado.index = range(df_setup_92_armado.shape[0])

print('======= ACABOU ===========')